# PTX and SASS (Nvidia)

## C Kernel to Source Code

In [2]:
import numpy as np
import pyopencl as cl
import pyopencl.array as cla

ctx = cl.create_some_context(answers=["nvi", 0])
queue = cl.CommandQueue(ctx)

In [3]:
prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

print(prg.binaries[0].decode())

//
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-35042711
// Unknown Toolkit Version
// Based on NVVM 7.0.1
//

.version 8.2
.target sm_70, texmode_independent
.address_size 64

	// .globl	sum

.entry sum(
	.param .u64 .ptr .global .align 4 sum_param_0,
	.param .u64 .ptr .global .align 4 sum_param_1,
	.param .u64 .ptr .global .align 4 sum_param_2
)
{
	.reg .f32 	%f<4>;
	.reg .b32 	%r<7>;
	.reg .b64 	%rd<8>;


	ld.param.u64 	%rd1, [sum_param_0];
	ld.param.u64 	%rd2, [sum_param_1];
	ld.param.u64 	%rd3, [sum_param_2];
	mov.b32 	%r1, %envreg3;
	mov.u32 	%r2, %ntid.x;
	mov.u32 	%r3, %ctaid.x;
	mov.u32 	%r4, %tid.x;
	add.s32 	%r5, %r4, %r1;
	mad.lo.s32 	%r6, %r2, %r3, %r5;
	mul.wide.s32 	%rd4, %r6, 4;
	add.s64 	%rd5, %rd1, %rd4;
	ld.global.f32 	%f1, [%rd5];
	add.s64 	%rd6, %rd2, %rd4;
	ld.global.f32 	%f2, [%rd6];
	add.f32 	%f3, %f1, %f2;
	add.s64 	%rd7, %rd3, %rd4;
	st.global.f32 	[%rd7], %f3;
	ret;

}

  


Comments:

* Intel or AT&T style?
* Note: address spaces always explicit
* What is `ctaid.x`? `%ntid.x`?
* How does parameter passing work?
* Is this the lowest-level abstraction?

In [10]:
!mkdir -p tmp

binary = prg.binaries[0]

with open("tmp/binary.ptx", "wb") as outf:
    outf.write(hacked_binary)
    
!(cd tmp; ptxas --gpu-name sm_70 --verbose binary.ptx -o binary.o)

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function 'sum' for 'sm_70'
ptxas info    : Function properties for sum
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 376 bytes cmem[0]


In [11]:
!cuobjdump --dump-sass tmp/binary.o


	code for sm_70
		Function : sum
	.headerflags	@"EF_CUDA_TEXMODE_INDEPENDENT EF_CUDA_64BIT_ADDRESS EF_CUDA_SM70 EF_CUDA_VIRTUAL_SM(EF_CUDA_SM70)"
        /*0000*/                   MOV R1, c[0x0][0x28] ;                 /* 0x00000a0000017a02 */
                                                                          /* 0x000fc40000000f00 */
        /*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;          /* 0x000000fffffff389 */
                                                                          /* 0x000fe200000e00ff */
        /*0020*/                   S2R R0, SR_TID.X ;                     /* 0x0000000000007919 */
                                                                          /* 0x000e220000002100 */
        /*0030*/                   MOV R7, 0x4 ;                          /* 0x0000000400077802 */
                                                                          /* 0x000fc60000000f00 */
        /*0040*/                   S2R R3, SR_CTAID.X ;      

## Is Division Expensive?

In [13]:
prg = cl.Program(ctx, """
__kernel void sum(
    __global float *a_g, int n)
{
    int gid = get_global_id(0);
    
    // try dividing by n
    
    int row = gid / 117;
    int col = gid % 117;
    
    a_g[row * 128 + col] *= 2;
    // a_g[gid] *= 2;
    
}
""").build()

binary = prg.binaries[0]

with open("tmp/binary.ptx", "wb") as outf:
    outf.write(hacked_binary)
    
!(cd tmp; ptxas --gpu-name sm_70 --verbose binary.ptx -o binary.o)
!cuobjdump --dump-sass tmp/binary.o | cut -c -80

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function 'sum' for 'sm_70'
ptxas info    : Function properties for sum
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 12 registers, 376 bytes cmem[0]

	code for sm_70
		Function : sum
	.headerflags	@"EF_CUDA_TEXMODE_INDEPENDENT EF_CUDA_64BIT_ADDRESS EF_CUDA_SM70 E
        /*0000*/                   MOV R1, c[0x0][0x28] ;                 /* 0x0
                                                                          /* 0x0
        /*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;          /* 0x0
                                                                          /* 0x0
        /*0020*/                   S2R R0, SR_TID.X ;                     /* 0x0
                                                                          /* 0x0
        /*0030*/                   MOV R7, 0x4 ;                          /* 0x0
                                                                  

## An Example with Control Flow

In [18]:
prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g, int n)
{
    int gsize = get_global_size(0);
    
    for (int i = get_global_id(0); i < n; i += gsize)
      res_g[i] = a_g[i] + b_g[i];
      
    res_g[get_global_id(0)] = 15;
}
""").build()

binary = prg.binaries[0]
#print(binary.decode())
with open("tmp/binary.ptx", "wb") as outf:
    outf.write(binary)
    
!(cd tmp; ptxas --gpu-name sm_70 --verbose binary.ptx -o binary.o)
!cuobjdump --dump-sass tmp/binary.o | cut -c -80

ptxas info    : 0 bytes gmem
ptxas info    : Compiling entry function 'sum' for 'sm_70'
ptxas info    : Function properties for sum
    0 bytes stack frame, 0 bytes spill stores, 0 bytes spill loads
ptxas info    : Used 28 registers, 380 bytes cmem[0]

	code for sm_70
		Function : sum
	.headerflags	@"EF_CUDA_TEXMODE_INDEPENDENT EF_CUDA_64BIT_ADDRESS EF_CUDA_SM70 E
        /*0000*/                   IMAD.MOV.U32 R1, RZ, RZ, c[0x0][0x28] ;      
                                                                                
        /*0010*/              @!PT SHFL.IDX PT, RZ, RZ, RZ, RZ ;                
                                                                                
        /*0020*/                   S2R R0, SR_TID.X ;                           
                                                                                
        /*0030*/                   BMOV.32.CLEAR RZ, B0 ;                       
                                                                  

* Spot something that doesn't quite seem to belong?

## From CUDA

Vector add stolen from [ORNL](https://www.olcf.ornl.gov/tutorials/cuda-vector-addition/).

In [20]:
%%writefile tmp/vector-add.cu

#include <stdio.h>
#include <stdlib.h>
#include <math.h>
 
__global__ void vecAdd(double *a, double *b, double *c, int n)
{
    // Get our global thread ID
    int id = blockIdx.x*blockDim.x+threadIdx.x;
 
    // Make sure we do not go out of bounds
    if (id < n)
        c[id] = a[id] + b[id];
}
 
int main( int argc, char* argv[] )
{
    // Size of vectors
    int n = 100000;
 
    double *h_a;
    double *h_b;
    double *h_c;
 
    double *d_a;
    double *d_b;
    double *d_c;
 
    size_t bytes = n*sizeof(double);
 
    h_a = (double*)malloc(bytes);
    h_b = (double*)malloc(bytes);
    h_c = (double*)malloc(bytes);
 
    cudaMalloc(&d_a, bytes);
    cudaMalloc(&d_b, bytes);
    cudaMalloc(&d_c, bytes);
 
    int i;
    for( i = 0; i < n; i++ ) {
        h_a[i] = sin(i)*sin(i);
        h_b[i] = cos(i)*cos(i);
    }
 
    // Copy host vectors to device
    cudaMemcpy( d_a, h_a, bytes, cudaMemcpyHostToDevice);
    cudaMemcpy( d_b, h_b, bytes, cudaMemcpyHostToDevice);
 
    int blockSize, gridSize;
 
    blockSize = 1024;
 
    gridSize = (int)ceil((float)n/blockSize);
 
    vecAdd<<<gridSize, blockSize>>>(d_a, d_b, d_c, n);
 
    cudaMemcpy( h_c, d_c, bytes, cudaMemcpyDeviceToHost );
 
    double sum = 0;
    for(i=0; i<n; i++)
        sum += h_c[i];
    printf("final result: %f\n", sum/n);
 
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);
 
    free(h_a);
    free(h_b);
    free(h_c);
 
    return 0;
}

Writing tmp/vector-add.cu


In [21]:
!(cd tmp; nvcc -c vector-add.cu)
!cuobjdump --dump-sass tmp/vector-add.o


Fatbin elf code:
arch = sm_52
code version = [1,7]
host = linux
compile_size = 64bit

	code for sm_52
		Function : _Z6vecAddPdS_S_i
	.headerflags	@"EF_CUDA_TEXMODE_UNIFIED EF_CUDA_64BIT_ADDRESS EF_CUDA_SM52 EF_CUDA_VIRTUAL_SM(EF_CUDA_SM52)"
                                                                                 /* 0x001cfc00e22007f6 */
        /*0008*/                   MOV R1, c[0x0][0x20] ;                        /* 0x4c98078000870001 */
        /*0010*/                   S2R R0, SR_CTAID.X ;                          /* 0xf0c8000002570000 */
        /*0018*/                   S2R R2, SR_TID.X ;                            /* 0xf0c8000002170002 */
                                                                                 /* 0x001fd842fec20ff1 */
        /*0028*/                   XMAD.MRG R3, R0.reuse, c[0x0] [0x8].H1, RZ ;  /* 0x4f107f8000270003 */
        /*0030*/                   XMAD R2, R0.reuse, c[0x0] [0x8], R2 ;         /* 0x4e00010000270002 */
        /*0038*/

* What is `_Z6vecAddPdS_S_i`?

In [36]:
!echo _Z6vecAddPdS_S_i | c++filt

vecAdd(double*, double*, double*, int)


## Inline PTX

In [22]:
prg = cl.Program(ctx, """
__kernel void getlaneid(__global int *d_ptr, int length)
{
    int elemID = get_global_id(0);
    if (elemID < length)
    {
        unsigned int laneid;
        asm("mov.u32 %0, %%laneid;" : "=r"(laneid));
        d_ptr[elemID] = laneid;
    }
}
""").build()

print(prg.binaries[0].decode())

//
// Generated by NVIDIA NVVM Compiler
//
// Compiler Build ID: CL-35042711
// Unknown Toolkit Version
// Based on NVVM 7.0.1
//

.version 8.2
.target sm_70, texmode_independent
.address_size 64

	// .globl	getlaneid

.entry getlaneid(
	.param .u64 .ptr .global .align 4 getlaneid_param_0,
	.param .u32 getlaneid_param_1
)
{
	.reg .pred 	%p<2>;
	.reg .b32 	%r<9>;
	.reg .b64 	%rd<4>;


	ld.param.u64 	%rd1, [getlaneid_param_0];
	ld.param.u32 	%r2, [getlaneid_param_1];
	mov.b32 	%r3, %envreg3;
	mov.u32 	%r4, %ctaid.x;
	mov.u32 	%r5, %ntid.x;
	mov.u32 	%r6, %tid.x;
	add.s32 	%r7, %r6, %r3;
	mad.lo.s32 	%r1, %r5, %r4, %r7;
	setp.ge.s32 	%p1, %r1, %r2;
	@%p1 bra 	$L__BB0_2;

	// begin inline asm
	mov.u32 %r8, %laneid;
	// end inline asm
	mul.wide.s32 	%rd2, %r1, 4;
	add.s64 	%rd3, %rd1, %rd2;
	st.global.u32 	[%rd3], %r8;

$L__BB0_2:
	ret;

}

  


* What do the constraints mean again?
* Spot the inline assembly
* Observe how the `if` is realized
* Observe the realization of `get_global_id()`

In [48]:
a = cla.empty(queue, 5000, np.uint32)
prg.getlaneid(queue, lanes.shape, None, a.data, np.uint32(a.size))

In [50]:
a[:500]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  0,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  0,  1,  2,  3,
        4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  0,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31,  0,  1,  2,  3,  4,  5,  6,  7,
        8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
       10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,
       27, 28, 29, 30, 31,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
       12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30, 31,  0,  1